# Setup

In [1]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False
# For colab
if IN_COLAB:
  import sys
  !git clone https://github.com/elements72/EDiReF-subtask-III.git
  %pip install lightning
  %pip install wandb
  sys.path.append('./EDiReF-subtask-III')
  %cd ./EDiReF-subtask-III
  num_workers = 2
else:
  num_workers = 0
  pass

In [2]:
# Reload modules, do not use cache
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
from pathlib import Path
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
import numpy as np
from utils import train_model, hyperparameters_tuning, train_model_seeds
import torch
from baselines import BertBaseline
from baselines import RandomUniformClassifier
from baselines import MajorityClassifier
import torch
from sklearn.model_selection import train_test_split
from attention import DialoguesAttention
from lstm_model import LSTMResModel
from dataset import MeldDataModule


Using device: cuda


In [4]:
import wandb
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: antonio-lopez (nlp-ediref). Use `wandb login --relogin` to force relogin


True

In [6]:
# reference can be retrieved in artifacts panel
# "VERSION" can be a version (ex: "v2") or an alias ("latest or "best")
checkpoint_reference = "nlp-ediref/EDiReF-subtask-III/model-8tqpz4hg:latest"

# download checkpoint locally (if not already cached)
run = wandb.init(project="EDiReF-subtask-III")
artifact = run.use_artifact(checkpoint_reference, type="model")
artifact_dir = artifact.download()

# load checkpoint
#model = LitModule.load_from_checkpoint(Path(artifact_dir) / "model.ckpt")

Thread SenderThread:
Traceback (most recent call last):
  File "/home/elements72/.local/lib/python3.10/site-packages/wandb/sdk/internal/internal_util.py", line 49, in run
    self._run()
  File "/home/elements72/.local/lib/python3.10/site-packages/wandb/sdk/internal/internal_util.py", line 100, in _run
    self._process(record)
  File "/home/elements72/.local/lib/python3.10/site-packages/wandb/sdk/internal/internal.py", line 328, in _process
    self._sm.send(record)
  File "/home/elements72/.local/lib/python3.10/site-packages/wandb/sdk/internal/sender.py", line 389, in send
    send_handler(record)
  File "/home/elements72/.local/lib/python3.10/site-packages/wandb/sdk/internal/sender.py", line 593, in send_exit
    self._update_summary()
  File "/home/elements72/.local/lib/python3.10/site-packages/wandb/sdk/internal/sender.py", line 1174, in _update_summary
    with open(summary_path, "w") as f:
FileNotFoundError: [Errno 2] No such file or directory: '/home/elements72/unibo/Vanno/NLP/

Problem at: /tmp/ipykernel_38135/3281128058.py 6 <module>


Traceback (most recent call last):
  File "/home/elements72/.local/lib/python3.10/site-packages/wandb/sdk/wandb_run.py", line 2211, in _atexit_cleanup
    self._on_finish()
  File "/home/elements72/.local/lib/python3.10/site-packages/wandb/sdk/wandb_run.py", line 2466, in _on_finish
    _ = exit_handle.wait(timeout=-1, on_progress=self._on_progress_exit)
  File "/home/elements72/.local/lib/python3.10/site-packages/wandb/sdk/lib/mailbox.py", line 298, in wait
    on_probe(probe_handle)
  File "/home/elements72/.local/lib/python3.10/site-packages/wandb/sdk/wandb_run.py", line 2428, in _on_probe_exit
    result = handle.wait(timeout=0, release=False)
  File "/home/elements72/.local/lib/python3.10/site-packages/wandb/sdk/lib/mailbox.py", line 281, in wait
    raise MailboxError("transport failed")
wandb.sdk.lib.mailbox.MailboxError: transport failed

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/elements72/.local/lib/p

Error: An unexpected error occurred

# Dataset

In [5]:
batch_size = 8
# Create and setup data module
datamodule = MeldDataModule(batch_size=batch_size, num_workers=num_workers)
datamodule.prepare_data()
datamodule.setup()
# Get dataloaders
train_loader, val_loader, test_loader = datamodule.train_dataloader(), datamodule.val_dataloader(), datamodule.test_dataloader()
# Get dataframes
train_data = datamodule.train_data
val_data = datamodule.val_data
test_data = datamodule.test_data
# Get encoder
emotion_encoder = datamodule.emotion_encoder

In [6]:
train_data.head(5)

,speakers,emotions,utterances,triggers
0,"[All, Ross, Phoebe, Rachel, Phoebe, Rachel, Ph...","[4, 4, 3, 6, 4, 1, 3, 6, 6, 1, 4, 4, 3, 4, 6]","[Hey., How's the maniac?, Oh, well he's yummy....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,"[Phoebe, Rachel, Phoebe, Phoebe, Phoebe, Phoeb...","[4, 6, 6, 4, 6, 4, 0, 4, 4, 5, 4]",[Okay. Would you rather live in the shirt pock...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,"[Ross, Emily, Ross, Emily, Ross, Emily, Emily,...","[3, 6, 3, 3, 5, 4, 4, 4, 2, 2, 2, 3, 4, 4]","[Emily!, Oh my God! What are you doing here?, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,"[Rachel, Barry, Rachel, Bernice, Barry, Rachel...","[4, 4, 0, 4, 4, 6, 3, 5]","[No, not that, I mean, what about you and Mind...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]"
4,"[Phoebe, Ross, Rachel, Monica, Rachel]","[3, 0, 3, 3, 0]","[Good for you! And hey, I thought your paper o...","[0.0, 0.0, 0.0, 0.0, 1.0]"


In [7]:
from sklearn.utils.class_weight import compute_class_weight

class_weights_emotion = compute_class_weight('balanced', classes=np.unique(train_data['emotions'].explode()),
                                             y=train_data['emotions'].explode())
class_weights_emotion = torch.tensor(class_weights_emotion, dtype=torch.float)

class_weights_trigger = compute_class_weight('balanced', classes=np.unique(train_data['triggers'].explode()), 
                                             y=train_data['triggers'].explode())
class_weights_trigger = torch.tensor(class_weights_trigger, dtype=torch.float)

class_weights_emotion

tensor([1.2499, 4.7433, 4.4692, 0.7979, 0.3270, 1.9099, 1.0752])

In [14]:
def get_hyperparams(model_name, freeze, lr):
    hyperparameters = {
        "lr": lr,
        "class_weights_emotion": class_weights_emotion,
        "class_weights_trigger": class_weights_trigger,
        "hidden_layers": 2,
        "use_encoder_cache": True if freeze else False,
        "encoder_cache_size": 100000,
        "freeze_bert": freeze,
        "dropout": 0.2,
        "bert_model_name": "bert-base-uncased",
        }
    if "lstm" in model_name:
        lstm_kwargs = {
            "hidden_size": 128,
            "num_layers": 2,
            "bidirectional": False,
        }
        hyperparameters["lstm_kwargs"] = lstm_kwargs
    if "emoberta" in model_name:
        hyperparameters["bert_model_name"] = "roberta-large"
    return hyperparameters


In [15]:
lr = 1e-3
epochs = 20
tune_hyperparameters = True
seeds = [6, 9, 42, 150, 300]

train_loader = datamodule.train_dataloader(collate_context=False)
val_loader = datamodule.train_dataloader(collate_context=False)

train_loader_emoberta = datamodule.train_dataloader(collate_context=True)
val_loader_emoberta = datamodule.train_dataloader(collate_context=True)


models = {
    "bert-baseline-freeze":{'model_class': BertBaseline, 'hyperparameters': get_hyperparams("bert-baseline-freeze", freeze=True, lr=lr), "train_loader": train_loader, "val_loader": val_loader},
    "bert-baseline-unfreeze":{'model_class': BertBaseline, 'hyperparameters': get_hyperparams("bert-baseline-unfreeze", freeze=False, lr=lr), "train_loader": train_loader, "val_loader": val_loader},
    "bert-lstm-freeze":{'model_class': LSTMResModel, 'hyperparameters': get_hyperparams("bert-lstm-freeze", freeze=True, lr=lr), "train_loader": train_loader, "val_loader": val_loader},
    "bert-lstm-unfreeze":{'model_class': LSTMResModel, 'hyperparameters': get_hyperparams("bert-lstm-unfreeze", freeze=False, lr=lr), "train_loader": train_loader, "val_loader": val_loader},
    "bert-attention-freeze":{'model_class': DialoguesAttention, 'hyperparameters': get_hyperparams("bert-attention-freeze", freeze=True, lr=lr), "train_loader": train_loader, "val_loader": val_loader},
    "bert-attention-unfreeze":{'model_class': DialoguesAttention, 'hyperparameters': get_hyperparams("bert-attention-unfreeze", freeze=False, lr=lr), "train_loader": train_loader, "val_loader": val_loader},
    "emoberta": {'model_class': BertBaseline, 'hyperparameters': get_hyperparams("emoberta", freeze=True, lr=lr), "train_loader": train_loader_emoberta, "val_loader": val_loader_emoberta},
    }


# Tune hyperparameters
if tune_hyperparameters:
    for model_name, model_dict in models.items():
        hyperparameters_tuning(model_dict['model_class'], model_name, datamodule, hyperparameters=model_dict['hyperparameters'])

    
for model_name, model_dict in models.items():
    train_model_seeds(**model_dict, model_name=model_name, seeds=seeds, epochs=epochs, wandb=wandb)

Using cache of size 100000 for the embedding


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacty of 1.95 GiB of which 2.38 MiB is free. Process 6041 has 1.69 GiB memory in use. Including non-PyTorch memory, this process has 266.00 MiB memory in use. Of the allocated memory 219.13 MiB is allocated by PyTorch, and 12.87 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
#Random Classifier
random_uniform_classifier = RandomUniformClassifier(7 , 1)
random_uniform_predictions_emotions = []
random_uniform_predictions_trigger = []
last_emotion = None
last_trigger= None
for batch in test_loader:
    emotions = batch["emotions"] 
    trigger = batch["triggers"]  
        
    #Emotions prediction

    predictions_emotions = random_uniform_classifier.predict_emotions(emotions)
    random_uniform_predictions_emotions.append(predictions_emotions)
        
    #Triggers prediction

    predictions_trigger = random_uniform_classifier.predict_triggers(trigger)
    random_uniform_predictions_trigger.append(predictions_trigger)
    

random_uniform_predictions_emotions = torch.cat(random_uniform_predictions_emotions, dim=0) 
random_uniform_predictions_trigger = torch.cat(random_uniform_predictions_trigger, dim =0) 

In [ ]:
#Majority Classifier                                 
majority_classifier = MajorityClassifier()        
majority_classifier.fit(train_loader)

majority_predictions_emotions = []
majority_predictions_trigger = []

for batch in test_loader:
    emotions = batch["emotions"]
    trigger = batch["triggers"]  
    
    #Emotions prediction
    predictions_emotions = majority_classifier.predict_emotions(emotions)
    majority_predictions_emotions.append(predictions_emotions)
    
    #Triggers prediction
    predictions_trigger = majority_classifier.predict_triggers(trigger)
    majority_predictions_trigger.append(predictions_trigger)

majority_predictions_emotions = torch.cat(majority_predictions_emotions)
majority_predictions_trigger = torch.cat(majority_predictions_trigger)